# Google Play Store Apps - Data Analysis Project
## Notebook 2: Data Cleaning and Preprocessing

### 1. Load Libraries and Data

In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load datasets
df_apps = pd.read_csv('googleplaystore.csv')
df_reviews = pd.read_csv('googleplaystore_user_reviews.csv')

print(f"Initial shape - Apps: {df_apps.shape}, Reviews: {df_reviews.shape}")

Initial shape - Apps: (10841, 13), Reviews: (34965, 5)


### 2. Handle Duplicate Rows

In [4]:
# Check for duplicates
print(f"Duplicate rows in apps: {df_apps.duplicated().sum()}")
print(f"Duplicate apps (by App name): {df_apps.duplicated(subset=['App']).sum()}")

# Keep first occurrence
df_apps_clean = df_apps.drop_duplicates(subset=['App'], keep='first')

print(f"Shape after removing duplicates: {df_apps_clean.shape}")

Duplicate rows in apps: 483
Duplicate apps (by App name): 1181
Shape after removing duplicates: (9660, 13)



### 3. Fix Data Type Issues

In [5]:
# Clean Rating column
df_apps_clean['Rating'] = pd.to_numeric(df_apps_clean['Rating'], errors='coerce')

# Clean Reviews column
df_apps_clean['Reviews'] = pd.to_numeric(df_apps_clean['Reviews'], errors='coerce')

# Clean Installs column (remove + and ,)
df_apps_clean['Installs'] = df_apps_clean['Installs'].str.replace('+', '').str.replace(',', '')
df_apps_clean['Installs'] = pd.to_numeric(df_apps_clean['Installs'], errors='coerce')

# Clean Price column (remove $)
df_apps_clean['Price'] = df_apps_clean['Price'].str.replace('$', '')
df_apps_clean['Price'] = pd.to_numeric(df_apps_clean['Price'], errors='coerce')

# Clean Size column
def clean_size(size):
    if pd.isna(size) or size == 'Varies with device':
        return np.nan
    elif 'M' in str(size):
        return float(str(size).replace('M', ''))
    elif 'k' in str(size):
        return float(str(size).replace('k', '')) / 1024
    else:
        return np.nan

df_apps_clean['Size'] = df_apps_clean['Size'].apply(clean_size)

print("✓ Data types fixed")
print(df_apps_clean.dtypes)

✓ Data types fixed
App                object
Category           object
Rating            float64
Reviews           float64
Size              float64
Installs          float64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


### 4. Handle Missing Values

In [6]:
# Analyze missing values
print("=== Missing Values After Cleaning ===")
missing = df_apps_clean.isnull().sum()
missing_pct = (missing / len(df_apps_clean) * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Strategy for handling missing values
# Rating: Drop if missing (usually indicates data quality issue)
# Type: Fill with mode
# Content Rating: Fill with mode
# Current Ver: Keep as is or drop
# Android Ver: Keep as is or drop

df_apps_clean = df_apps_clean.dropna(subset=['Rating'])
df_apps_clean['Type'].fillna(df_apps_clean['Type'].mode()[0], inplace=True)
df_apps_clean['Content Rating'].fillna(df_apps_clean['Content Rating'].mode()[0], inplace=True)

print(f"\nShape after handling missing values: {df_apps_clean.shape}")

=== Missing Values After Cleaning ===
                Missing_Count  Percentage
Rating                   1463       15.14
Size                     1228       12.71
Current Ver                 8        0.08
Android Ver                 3        0.03
Reviews                     1        0.01
Content Rating              1        0.01
Price                       1        0.01
Installs                    1        0.01
Type                        1        0.01

Shape after handling missing values: (8197, 13)


### 5. Remove Outliers and Invalid Data

In [7]:
# Remove invalid ratings (outside 0-5 range)
df_apps_clean = df_apps_clean[(df_apps_clean['Rating'] >= 0) & (df_apps_clean['Rating'] <= 5)]

# Remove rows with extreme outliers in Reviews (optional)
Q1 = df_apps_clean['Reviews'].quantile(0.25)
Q3 = df_apps_clean['Reviews'].quantile(0.75)
IQR = Q3 - Q1
# We'll keep outliers for business analysis but flag them

print(f"Valid rating range: {df_apps_clean['Rating'].min()} to {df_apps_clean['Rating'].max()}")
print(f"Final shape: {df_apps_clean.shape}")

Valid rating range: 1.0 to 5.0
Final shape: (8196, 13)


### 6. Feature Engineering

In [8]:
# Create new features

# 1. App Age (if Last Updated is available)
df_apps_clean['Last Updated'] = pd.to_datetime(df_apps_clean['Last Updated'], errors='coerce')
current_date = pd.to_datetime('2018-08-10')  # Approximate dataset date
df_apps_clean['Days_Since_Update'] = (current_date - df_apps_clean['Last Updated']).dt.days

# 2. Price Category
df_apps_clean['Price_Category'] = df_apps_clean['Price'].apply(
    lambda x: 'Free' if x == 0 else ('Budget' if x < 5 else ('Premium' if x < 20 else 'Expensive'))
)

# 3. Install Range
def categorize_installs(installs):
    if installs < 1000:
        return 'Low'
    elif installs < 100000:
        return 'Medium'
    elif installs < 10000000:
        return 'High'
    else:
        return 'Very High'

df_apps_clean['Install_Range'] = df_apps_clean['Installs'].apply(categorize_installs)

# 4. Rating Category
df_apps_clean['Rating_Category'] = pd.cut(
    df_apps_clean['Rating'],
    bins=[0, 2, 3, 4, 5],
    labels=['Poor', 'Average', 'Good', 'Excellent']
)

print("✓ New features created")
print(df_apps_clean[['Price_Category', 'Install_Range', 'Rating_Category']].head())

✓ New features created
  Price_Category Install_Range Rating_Category
0           Free        Medium       Excellent
1           Free          High            Good
2           Free          High       Excellent
3           Free     Very High       Excellent
4           Free          High       Excellent


### 7. Clean Reviews Dataset

In [9]:
# Handle missing values in sentiment data
print("=== Reviews Dataset Cleaning ===")
print(f"Missing values:\n{df_reviews.isnull().sum()}")

# Drop rows where all sentiment columns are missing
df_reviews_clean = df_reviews.dropna(subset=['Translated_Review'], how='all')

# Fill sentiment scores with 0 (neutral) where missing
sentiment_cols = ['Sentiment_Polarity', 'Sentiment_Subjectivity']
df_reviews_clean[sentiment_cols] = df_reviews_clean[sentiment_cols].fillna(0)

print(f"Reviews shape after cleaning: {df_reviews_clean.shape}")

=== Reviews Dataset Cleaning ===
Missing values:
App                           0
Translated_Review         14590
Sentiment                 14587
Sentiment_Polarity        14587
Sentiment_Subjectivity    14587
dtype: int64
Reviews shape after cleaning: (20375, 5)


### 8. Data Validation

In [10]:
# Verify data quality after cleaning
print("=== Data Quality Checks ===")

# Check for remaining nulls in critical columns
critical_cols = ['App', 'Category', 'Rating', 'Reviews', 'Installs', 'Type', 'Price']
for col in critical_cols:
    null_count = df_apps_clean[col].isnull().sum()
    print(f"{col}: {null_count} nulls")

# Check data ranges
print(f"\nRating range: {df_apps_clean['Rating'].min():.2f} - {df_apps_clean['Rating'].max():.2f}")
print(f"Price range: ${df_apps_clean['Price'].min():.2f} - ${df_apps_clean['Price'].max():.2f}")
print(f"Installs range: {df_apps_clean['Installs'].min():.0f} - {df_apps_clean['Installs'].max():.0f}")

# Check category distribution
print(f"\nCategories: {df_apps_clean['Category'].nunique()} unique")
print(df_apps_clean['Category'].value_counts().head())

=== Data Quality Checks ===
App: 0 nulls
Category: 0 nulls
Rating: 0 nulls
Reviews: 0 nulls
Installs: 0 nulls
Type: 0 nulls
Price: 0 nulls

Rating range: 1.00 - 5.00
Price range: $0.00 - $400.00
Installs range: 1 - 1000000000

Categories: 33 unique
Category
FAMILY          1608
GAME             912
TOOLS            718
FINANCE          302
PRODUCTIVITY     301
Name: count, dtype: int64


### 9. Save Cleaned Data

In [11]:
# Save cleaned datasets
df_apps_clean.to_csv('apps_cleaned.csv', index=False)
df_reviews_clean.to_csv('reviews_cleaned.csv', index=False)

print("\n✓ Data cleaning complete!")
print(f"✓ Clean apps dataset: {df_apps_clean.shape}")
print(f"✓ Clean reviews dataset: {df_reviews_clean.shape}")
print("✓ Files saved: apps_cleaned.csv, reviews_cleaned.csv")


✓ Data cleaning complete!
✓ Clean apps dataset: (8196, 17)
✓ Clean reviews dataset: (20375, 5)
✓ Files saved: apps_cleaned.csv, reviews_cleaned.csv
